In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Transmute.AI/ChipNet/ChipNet-master

/content/drive/MyDrive/Transmute.AI/ChipNet/ChipNet-master


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Transmute.AI/ChipNet/ChipNet-master')

In [4]:
import argparse
import os

import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm as tqdm_notebook
from datasets import DataManager
from utils import *
from models import get_model

In [5]:
seed_everything(43)

In [7]:
dataset = 'c100'
model = 'r164'
budget_type = 'channel_ratio'
Vc = 0.5
batch_size = 128
epochs = 10
name = "r164"
host_name = None
valid_size = 0.1
lr = 0.05
scheduler_type = 1
decay = 0.001
test_only = False
workers = 0
cuda_id = 0

In [14]:
vc = 0.25
vc = torch.FloatTensor([vc])

In [9]:
Vc = torch.FloatTensor([Vc])
if host_name == None:
    model_path = f"checkpoints/r164_c10_{str(np.round(vc.item(),decimals=6))}_channel_ratio_pruned.pth"
else:
#     model_path = f"checkpoints/{args.name}_pretrained.pth"
    model_path = f"checkpoints/{host_name}_pruned.pth"

In [10]:
data_object = DataManager(dataset, batch_size, workers, valid_size)
trainloader, valloader, testloader = data_object.prepare_data()
dataloaders = {
        'train': trainloader, 'val': valloader, "test": testloader
}

... Preparing data ...
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
using fixed split
45000 5000


In [11]:
model = get_model(model, 'prune', data_object.num_classes, data_object.insize)
if host_name is not None:
    host_state = torch.load(model_path)['state_dict']
    model.load_state_dict(get_mask_dict(model.state_dict(), host_state), strict = False)
else:
    state = torch.load(model_path)['state_dict']
    model.load_state_dict(state, strict=False)
CE = nn.CrossEntropyLoss()
def criterion(model, y_pred, y_true):
    ce_loss = CE(y_pred, y_true)
    return ce_loss

RuntimeError: ignored

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=decay)
device = torch.device(f"cuda:{str(cuda_id)}")
model.to(device)
Vc.to(device)

In [ ]:
def train(model, loss_fn, optimizer):
    model.train()
    counter = 0
    tk1 = tqdm_notebook(dataloaders['train'], total=len(dataloaders['train']))
    running_loss = 0.
    for x_var, y_var in tk1:
        counter +=1
        x_var = x_var.to(device=device)
        y_var = y_var.to(device=device)
        scores = model(x_var)
        loss = loss_fn(model, scores, y_var)
        running_loss+=loss.item()
        tk1.set_postfix(loss=running_loss/counter)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return running_loss/counter    

In [ ]:
def test(model, loss_fn, optimizer, phase):
    model.eval()
    counter = 0
    tk1 = tqdm_notebook(dataloaders[phase], total=len(dataloaders[phase]))
    running_loss = 0
    running_acc = 0
    total = 0
    with torch.no_grad():
        for x_var, y_var in tk1:
            counter +=1
            x_var = x_var.to(device=device)
            y_var = y_var.to(device=device)
            scores = model(x_var)
            loss = loss_fn(model, scores, y_var)
            _, scores = torch.max(scores.data, 1)
            y_var = y_var.cpu().detach().numpy()
            scores = scores.cpu().detach().numpy()

            correct = (scores == y_var).sum().item()
            running_loss+=loss.item()
            running_acc+=correct
            total+=scores.shape[0]
            tk1.set_postfix(loss=running_loss/counter, acc=running_acc/total)
    return running_acc/total, running_loss/counter

In [ ]:
model.prepare_for_finetuning(device, Vc.item(), budget_type=budget_type) # sets beta and gamma and unfreezes network except zetas

In [ ]:
best_accuracy=0
num_epochs = epochs
train_losses = []
valid_losses = []
valid_accuracy = []
if test_only == False:
    for epoch in range(num_epochs):
        adjust_learning_rate(optimizer, epoch, scheduler_type, lr, epochs)
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        train_loss = train(model, criterion, optimizer)
        accuracy, valid_loss = test(model, criterion, optimizer, "val")
        remaining = model.get_remaining(20.,budget_type).item()
        
        if accuracy>best_accuracy:
            print("**Saving model**")
            best_accuracy=accuracy
            torch.save({
                "epoch": epoch + 1,
                "state_dict" : model.state_dict(),
                "acc" : best_accuracy,
                "rem" : remaining,
            }, f"checkpoints/r164_c10_finetuned.pth")
            
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        valid_accuracy.append(accuracy)
        df_data=np.array([train_losses, valid_losses, valid_accuracy]).T
        df = pd.DataFrame(df_data,columns = ['train_losses','valid_losses','valid_accuracy'])
        df.to_csv(f"logs/r164_c10_finetuned.csv")

Starting epoch 1 / 50


100%|██████████| 40/40 [00:09<00:00,  4.33it/s, acc=0.449, loss=2.07]


**Saving model**
Starting epoch 2 / 50


100%|██████████| 40/40 [00:09<00:00,  4.29it/s, acc=0.496, loss=1.8]


**Saving model**
Starting epoch 3 / 50


...

100%|██████████| 40/40 [00:09<00:00,  4.37it/s, acc=0.604, loss=1.42]


Starting epoch 41 / 50


100%|██████████| 40/40 [00:09<00:00,  4.37it/s, acc=0.601, loss=1.4]


Starting epoch 42 / 50


 87%|████████▋ | 305/352 [05:12<00:48,  1.03s/it, loss=1.09]

In [ ]:
state = torch.load(f"checkpoints/r164_c10_finetuned.pth")
model.load_state_dict(state['state_dict'],strict=True)
acc, v_loss = test(model, criterion, optimizer, "test")
print(f"Test Accuracy: {acc} | Valid Accuracy: {state['acc']}")